<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">TFM</p>
<p style="margin: 0; text-align:right;">Master universitari en Ciència de dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis de Informàtica, Multimedia i Telecomunicació</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# TFM

data: https://drive.google.com/drive/folders/1fx_j6gmiATvhEaBtUwEPuCOl8peJ2zDM

code source: https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/

## 1. Simple k-NN implementation

### Euclidean distance

In [1]:
from math import sqrt
import pandas as pd
from random import seed
from random import randrange
from IPython.core.debugger import Pdb;

In [2]:
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]

In [3]:
# calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

In [ ]:
row0 = dataset[0]
for row in dataset:
    distance = euclidean_distance(row0, row)
    #print(distance)

### Get nearest neighbors

In [4]:
# Locate the most similar neighbors
def get_neighbors(train, test_row, k):
    distances = list()
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(k):
        neighbors.append(distances[i][0])
    return neighbors

In [17]:
neighbors = get_neighbors(dataset, dataset[0], 3)
for neighbor in neighbors:
    print(neighbor)

[2.7810836, 2.550537003, 0]
[3.06407232, 3.005305973, 0]
[1.465489372, 2.362125076, 0]


### Predictions

In [18]:
# Make a classification prediction with neighbors
def predict_classification(train, test_row, k):
    neighbors = get_neighbors(train, test_row, k)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

In [19]:
prediction = predict_classification(dataset, dataset[5], 3)
print('Expected %d, Got %d.' % (dataset[5][-1], prediction))

Expected 1, Got 1.


### Data cleaning

In [7]:
df = pd.read_csv('data/DSI1_trncrd.csv', header=None)
df.head()

,0,1,2,3,4
0,73.756532,54.633661,3.7,1,1
1,73.756532,54.633661,3.7,1,1
2,73.756532,54.633661,3.7,1,1
3,76.199587,53.462735,3.7,1,1
4,76.199587,53.462735,3.7,1,1


In [8]:
# for now, drop columns of 'floor' and 'building', keep x,y,z
df.drop(df.columns[[3,4]], axis=1, inplace=True)
df.head()

,0,1,2
0,73.756532,54.633661,3.7
1,73.756532,54.633661,3.7
2,73.756532,54.633661,3.7
3,76.199587,53.462735,3.7
4,76.199587,53.462735,3.7


In [9]:
print(df.duplicated().sum())
print(df.shape[0])

1139
1369


In [10]:
# dropping ALL duplicate values
df.drop_duplicates(keep = 'first', inplace = True)
print(df.duplicated().sum())
df.head()

0


,0,1,2
0,73.756532,54.633661,3.7
3,76.199587,53.462735,3.7
9,78.208971,54.113250,3.7
15,78.743835,56.006966,3.7
21,89.441223,63.336084,3.7


### Algorithm

In [11]:
# kNN Algorithm
def k_nearest_neighbors(train, test, k):
    predictions = list()
    for row in test:
        output = predict_classification(train, row, k)
        predictions.append(output)
    return(predictions)

In [12]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = dataset.values.tolist()
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [16]:
# Test the kNN on the example dataset
seed(1)
# evaluate algorithm
n_folds = 5
k = 5
#Pdb().set_trace()
scores = evaluate_algorithm(df, k_nearest_neighbors, n_folds, k)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
#label = predict_classification(dataset, row, num_neighbors)
#print('Data=%s, Predicted: %s' % (row, label))

Scores: [100.0, 100.0, 100.0, 100.0, 100.0]
Mean Accuracy: 100.000%


In [ ]:
row = [76.199587,53.462735,3.7]
prediction = predict_classification(dataset, row, 3)
print('Expected %d, Got %d.' % (row[-1], prediction))